# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770831422053                   -0.53    8.0
  2   -2.772140124772       -2.88       -1.31    1.0
  3   -2.772170111817       -4.52       -2.58    1.0
  4   -2.772170715344       -6.22       -3.71    2.0
  5   -2.772170722345       -8.15       -4.11    2.0
  6   -2.772170723002       -9.18       -4.99    1.0
  7   -2.772170723014      -10.93       -5.77    1.0
  8   -2.772170723015      -11.94       -6.30    2.0
  9   -2.772170723015      -14.03       -6.69    1.0
 10   -2.772170723015      -14.45       -7.09    1.0
 11   -2.772170723015   +  -15.35       -7.90    2.0
 12   -2.772170723015      -13.92       -8.55    1.0
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770789244638                   -0.52    9.0
  2   -2.772060985538       -2.90       -1.32    1.0
  3   -2.772083109637       -4.66       -2.47 

1.773558037674741

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -2.770771102104                   -0.52    9.0
  2   -2.772060864673       -2.89       -1.32    1.0
  3   -2.772083027561       -4.65       -2.44    1.0
  4   -2.772083338213       -6.51       -3.13    1.0
  5   -2.772083417490       -7.10       -4.22    2.0
  6   -2.772083417702       -9.68       -4.55    1.0
  7   -2.772083417809       -9.97       -5.69    1.0
  8   -2.772083417811      -11.93       -6.07    2.0
  9   -2.772083417811      -13.42       -6.43    1.0
 10   -2.772083417811      -13.89       -8.25    1.0

Polarizability via ForwardDiff:       1.7725349319705532
Polarizability via finite difference: 1.773558037674741
